# Contract Analysis System with CrewAI

### Usage Instructions
- Place your contract PDF in the notebook directory
- Set up .env file with OpenAI API key
- un all cells in sequence
- View the boxen-formatted output for:
  - Agent profiles
  - Task details
  - Analysis results


### Setup and Dependencies

In [ ]:
# Install required packages
! pip install crewai pyboxen langchain python-dotenv PyPDF2 agentops openai


# Setting Up AI Agent Environment

## Required Imports

First, we need to import all necessary libraries for our AI agent workflow:

In [10]:
# Import necessary libraries and modules

# Load environment variables from .env file
from dotenv import load_dotenv

# Operating system interface
import os

# CrewAI components for agent orchestration
from crewai import Crew, Agent

# Pretty printing tool for terminal output
from pyboxen import boxen

# Tool for monitoring and logging AI agent operations
import agentops

# LangChain's ChatOpenAI model integration
from langchain.chat_models import ChatOpenAI

# Initialize environment variables from .env file
# This will look for a .env file in the current directory
# and load any variables defined there into the environment
load_dotenv()

True

# Environment and Monitoring Setup

## AgentOps Initialization
- Initialize AgentOps monitoring system and setup agnetop api key and open api key in the env file.
- Set tag 'contract-analysis' for tracking specific workflow
- Enables logging and performance monitoring of AI agents
- You get the AgentOP AI Dashboard Session Link From :
- Example : 2024-12-24 19:53:24,323 - 22992 - client.py-client:258 - INFO: View info on this session at https://app.agentops.ai/drilldown?session_id=8d76b2ea-de2f-424b-afe8-afd267d95b6b

#### Env File :
- OPENAI_API_KEY=
- AGENTOPS_API_KEY=
- AGENTOPS_PARENT_KEY=

In [11]:
# Initialize AgentOps for monitoring
agentops.init(tags=['contract-analysis'])

# Load OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

2024-12-24 19:53:24,323 - 22992 - client.py-client:258 - INFO: View info on this session at https://app.agentops.ai/drilldown?session_id=8d76b2ea-de2f-424b-afe8-afd267d95b6b


### Reference Data Configuration

In [12]:
reference_data = {
    "limitation of liability": "30 days",
    "owner expiry date": "30 days",
    "notice period": "30 days",
    "agreement duration": "12 months",
    "payment terms": "30 days",
    "confidentiality period": "3 years",
    "insurance coverage": "$1,000,000",
    "maximum monthly hours": "120"
}

#### Showing Reference Data Log

In [13]:
print(boxen(
    "\n".join([f"{k}: {v}" for k, v in reference_data.items()]),
    title="📋 Reference Contract Terms",
    padding=1,
    margin=1,
    color="blue"
))

                                                                                                                   
   ╭─ 📋 Reference Contract Terms ────────╮                                                                        
   │                                      │                                                                        
   │   limitation of liability: 30 days   │                                                                        
   │   owner expiry date: 30 days         │                                                                        
   │   notice period: 30 days             │                                                                        
   │   agreement duration: 12 months      │                                                                        
   │   payment terms: 30 days             │                                                                        
   │   confidentiality period: 3 years    │                              

# Custom LangChain Tools Implementation

## PDF Reader Tool
This custom tool extends LangChain's BaseTool to provide PDF processing capabilities.

### Features:
- Reads PDF files from specified file path
- Extracts text from all pages
- Handles errors gracefully with informative messages

In [14]:
# Import required tools and PDF processing libraries
from langchain_community.tools import BaseTool
from PyPDF2 import PdfReader

# Custom PDF Reading Tool
class PdfReadTool(BaseTool):
    """Tool for reading and extracting text from PDF files"""
    name = "PDF Reader"
    description = "Read PDF files"

    def _run(self, file_path: str) -> str:
        try:
            # Open and process the PDF file
            with open(file_path, 'rb') as file:
                reader = PdfReader(file)
                text = ""
                # Extract text from each page
                for page in reader.pages:
                    text += page.extract_text()
            return text
        except Exception as e:
            return f"Error: {str(e)}"

# Reference Data Access Tool        
class ReferenceDataTool(BaseTool):
    """Tool for accessing standard reference data"""
    name = "Reference Data Tool"
    description = "Access the standard reference data for comparison"

    def _run(self, action_input: str = None) -> dict:
        return reference_data

# Agent Creation and Configuration

## Function: `create_agents()`
Creates and configures three specialized AI agents for contract analysis.
- Researcher_agent
- Writer Agent
- Review Agent

In [15]:
def create_agents():
   # Create PDF Data Extractor Agent
   researcher_agent = Agent(
       role='PDF Data Extractor',
       goal='Extract key contract terms from PDF',
       backstory='Expert in extracting contract terms from PDFs',
       llm=ChatOpenAI(temperature=0.7, model='gpt-4'),
       verbose=True,
       tools=[PdfReadTool()]
   )
   
   # Create Data Comparison Analyst Agent
   writer_agent = Agent(
       role='Data Comparison Analyst',
       goal='Compare contract terms with reference values',
       backstory='Expert in analyzing contract terms',
       llm=ChatOpenAI(temperature=0.7, model='gpt-4'),
       verbose=True,
       tools=[ReferenceDataTool()]
   )
   
   # Create Compliance Report Generator Agent
   review_agent = Agent(
       role='Compliance Report Generator',
       goal='Create detailed report of contract deviations',
       backstory='Expert in compliance reporting',
       llm=ChatOpenAI(temperature=0.7, model='gpt-4'),
       verbose=True,
       tools=[ReferenceDataTool()]
   )
   
   # Display formatted information for each agent
   for agent in [researcher_agent, writer_agent, review_agent]:
       print(boxen(
           f"Role: {agent.role}\nGoal: {agent.goal}\nBackstory: {agent.backstory}",
           title="🤖 Agent Profile",
           padding=1,
           margin=1,
           color="green"
       ))
   
   return researcher_agent, writer_agent, review_agent

# Initialize all agents
researcher_agent, writer_agent, review_agent = create_agents()

c:\Users\sachi\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


                                                                                                                   
   ╭─ 🤖 Agent Profile ───────────────────────────────────────────╮                                                
   │                                                              │                                                
   │   Role: PDF Data Extractor                                   │                                                
   │   Goal: Extract key contract terms from PDF                  │                                                
   │   Backstory: Expert in extracting contract terms from PDFs   │                                                
   │                                                              │                                                
   ╰──────────────────────────────────────────────────────────────╯                                                
                                                                         

                                                                                                                   
   ╭─ 🤖 Agent Profile ─────────────────────────────────────╮                                                      
   │                                                        │                                                      
   │   Role: Data Comparison Analyst                        │                                                      
   │   Goal: Compare contract terms with reference values   │                                                      
   │   Backstory: Expert in analyzing contract terms        │                                                      
   │                                                        │                                                      
   ╰────────────────────────────────────────────────────────╯                                                      
                                                                         

                                                                                                                   
   ╭─ 🤖 Agent Profile ──────────────────────────────────────╮                                                     
   │                                                         │                                                     
   │   Role: Compliance Report Generator                     │                                                     
   │   Goal: Create detailed report of contract deviations   │                                                     
   │   Backstory: Expert in compliance reporting             │                                                     
   │                                                         │                                                     
   ╰─────────────────────────────────────────────────────────╯                                                     
                                                                         

# Task Creation and Configuration

## Function: `create_tasks()`
Defines the workflow tasks for contract analysis using CrewAI's Task framework.

### Task Components
- PDF Data Extraction Task
- Comparison Task
- Final Report Task

#### Note - Add Your PDF File Here: 
### Initialize tasks with input file
input_file = "contract.pdf"


In [21]:
from crewai import Task

def create_tasks(researcher_agent, writer_agent, review_agent, input_file):
   # Import textwrap for clean multiline strings
   from textwrap import dedent
   
   # Task 1: PDF Data Extraction
   pdf_extraction_task = Task(
       description=dedent(f"""Use the PDF Reader tool to read {input_file} and extract key terms."""),
       agent=researcher_agent,
       expected_output="Dictionary containing extracted contract terms"
   )
   
   # Task 2: Data Comparison
   comparison_task = Task(
       description=dedent(f"""Compare extracted terms with reference data and identify mismatches."""),
       agent=writer_agent,
       expected_output="List of mismatches between extracted and reference data"
   )
   
   # Task 3: Report Generation
   final_report_task = Task(
       description=dedent(f"""Create comprehensive analysis report with findings."""),
       agent=review_agent,
       expected_output="Detailed analysis report with findings and recommendations"
   )
   
   # Display formatted information for each task
   for task in [pdf_extraction_task, comparison_task, final_report_task]:
       print(boxen(
           f"Description: {task.description}\nExpected Output: {task.expected_output}",
           title="📝 Task Details",
           padding=1,
           margin=1,
           color="yellow"
       ))
   
   return pdf_extraction_task, comparison_task, final_report_task

# Initialize tasks with input file
input_file = "contract.pdf"
tasks = create_tasks(researcher_agent, writer_agent, review_agent, input_file)

                                                                                                                   
   ╭─ 📝 Task Details ────────────────────────────────────────────────────────────────────╮                        
   │                                                                                      │                        
   │   Description: Use the PDF Reader tool to read contract.pdf and extract key terms.   │                        
   │   Expected Output: Dictionary containing extracted contract terms                    │                        
   │                                                                                      │                        
   ╰──────────────────────────────────────────────────────────────────────────────────────╯                        
                                                                                                                   



                                                                                                                   
   ╭─ 📝 Task Details ─────────────────────────────────────────────────────────────────────╮                       
   │                                                                                       │                       
   │   Description: Compare extracted terms with reference data and identify mismatches.   │                       
   │   Expected Output: List of mismatches between extracted and reference data            │                       
   │                                                                                       │                       
   ╰───────────────────────────────────────────────────────────────────────────────────────╯                       
                                                                                                                   



                                                                                                                   
   ╭─ 📝 Task Details ───────────────────────────────────────────────────────────────╮                             
   │                                                                                 │                             
   │   Description: Create comprehensive analysis report with findings.              │                             
   │   Expected Output: Detailed analysis report with findings and recommendations   │                             
   │                                                                                 │                             
   ╰─────────────────────────────────────────────────────────────────────────────────╯                             
                                                                                                                   



# 📜 Contract Analysis System with CrewAI

This script is designed to run an automated contract analysis process using a fictional or proprietary library called **CrewAI**. Below is an in-depth breakdown of the code and its functionality.

---

## **Function: `run_analysis`**

In [25]:
# Contract Analysis System with CrewAI
def run_analysis(agents, tasks):
    crew = Crew(
        agents=agents,
        tasks=tasks,
        verbose=True
    )
    
    result = crew.kickoff()
    
    # Display final results with supported parameters only
    print(boxen(
        result,
        title="📊 Contract Analysis Report",
        padding=1,
        margin=1,
        color="magenta"
    ))
    
    return result

# Execute analysis
final_report = run_analysis([researcher_agent, writer_agent, review_agent], tasks)


2024-12-24 21:06:11,524 - 22992 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: PDF Data Extractor
 [INFO]: == Starting Task: Use the PDF Reader tool to read contract.pdf and extract key terms.


> Entering new CrewAgentExecutor chain...
First, I need to use the PDF Reader tool to read the contract.pdf and extract the key terms.

Action: 
PDF Reader

Action Input: 
{
  "file": "contract.pdf"
}
 

CONSULTING SERVICES AGREEMENT  
 
This Consulting Services Agreement ("Agreement") is made effective as of 15/01/2024, by and between:  
 
ABC Technologies Inc.  
123 Tech Park Avenue  
Silicon Valley, CA 94025  
 
Contact: john.doe@abctech.com  
 
and 
 
XYZ Consulting LLC ("Consultant")  
456 Business Plaza  
New York, NY 10001  
 
KEY TERMS AND CONDITIONS:  
 
1. Agreement Duration:  
   Start Date: 15/01/2024  
   End Date: 14/01/2025  
 
2. Service Scope:  
   - Technical consulting  
   - Project management  
   - Strategic advisory  
 
3. Financial Terms:  
   - Rate: $150 per hour     - Payment terms: Net 45 days  
   - Maximum monthly 

                                                                                                                   
   ╭─ 📊 Contract Analysis Report ─────────────────────────────────────────────────────────────────────────────╮   
   │                                                                                                           │   
   │   Compliance Report:                                                                                      │   
   │                                                                                                           │   
   │   Title: Analysis of Contract Deviations                                                                  │   
   │                                                                                                           │   
   │   Introduction:                                                                                           │   
   │   This report presents an analysis of the discrepancies identified b